In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist #cifar 같은거 써도 됨
#tf.enable_eager_execution()
from time import time
import os
print(tf.__version__)

def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")

    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt :
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0

def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0

    return train_data, test_data

def load_mnist() :
    (train_data, train_labels), (test_data, test_labels) = mnist.load_data()
    train_data = np.expand_dims(train_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1] : batch_size, height, width, channel
    test_data = np.expand_dims(test_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]

    train_data, test_data = normalize(train_data, test_data) #0~255 -> 0~1

    train_labels = to_categorical(train_labels, 10) # [N,] -> [N, 10] #one hot incoding
    test_labels = to_categorical(test_labels, 10) # [N,] -> [N, 10]

    return train_data, train_labels, test_data, test_labels

def flatten() :
    return tf.keras.layers.Flatten()

def dense(channel, weight_init) :
    return tf.keras.layers.Dense(units = channel, use_bias = True, kernel_initializer=weight_init)

def sigmoid() :
    return tf.keras.layers.Activation(tf.keras.activations.sigmoid)

def relu():
    return tf.keras.layers.Activation(tf.keras.activations.relu)

def dropout(rate) : 
    return tf.keras.layers.Dropout(rate)

2.3.0


In [5]:
# Class 

# class create_model_class(tf.keras.Model):
#     def __init__(self, label_dim):
#         super(create_model_class, self).__init__()
#         weight_init = tf.keras.initializers.glorot_uniform()

#         self.model = tf.keras.Sequential()
#         self.model.add(flatten())

#         for i in range(2):
#             self.model.add(dense(256, weight_init))
#             self.model.add(relu())
#             self.model.add(dropout(rate=0.5)) #각 뉴런의 50%만 사용하여 학습 진행

#         self.model.add(dense(label_dim, weight_init))

#     def call(self, x, training=None, mask=None):
#         x = self.model(x)
#         return x

#Function

def create_model_function(label_dim) :
    weight_init = tf.keras.initializers.glorot_uniform()

    model = tf.keras.Sequential()
    model.add(flatten())

    for i in range(2) :
        model.add(dense(256, weight_init))
        #model.add(sigmoid())
        model.add(relu())
        model.add(dropout(rate=0.5))

    model.add(dense(label_dim, weight_init))

    return model

def loss_fn (model, images, labels) : 
    logits = model(images, training = True)
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred = logits, y_true = labels, from_logits = True))
    return loss

def accuracy_fn(model, images, labels):
    logits = model(images, training = False)
    prediction = tf.equal(tf.argmax(labels, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)

In [6]:
""" dataset """
train_x, train_y, test_x, test_y = load_mnist()

""" parameters """
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

""" Graph Input using Dataset API """
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=batch_size).\
    batch(batch_size, drop_remainder=True)

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=len(test_x)).\
    batch(len(test_x))

""" Model """
network = create_model_function(label_dim)

""" Training """
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

""" Writer """
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_dropout'

checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

In [7]:
if train_flag :

    checkpoint = tf.train.Checkpoint(dnn=network)

    # create writer for tensorboard
    summary_writer = tf.summary.create_file_writer(logdir=logs_dir)
    start_time = time()

    # restore check-point if it exits
    could_load, checkpoint_counter = load(network, checkpoint_dir)    

    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)        
        counter = checkpoint_counter        
        print(" [*] Load SUCCESS")
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(" [!] Load failed...")
    
    # train phase
    with summary_writer.as_default():  # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx, (train_input, train_label) in enumerate(train_dataset):            
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network.variables))

                train_loss = loss_fn(network, train_input, train_label)
                train_accuracy = accuracy_fn(network, train_input, train_label)
                
                for test_input, test_label in test_dataset:                
                    test_accuracy = accuracy_fn(network, test_input, test_label)

                tf.summary.scalar(name='train_loss', data=train_loss, step=counter)
                tf.summary.scalar(name='train_accuracy', data=train_accuracy, step=counter)
                tf.summary.scalar(name='test_accuracy', data=test_accuracy, step=counter)

                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy,
                       test_accuracy))
                counter += 1                
        checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))
        
# test phase      
else :
    _, _ = load(network, checkpoint_dir)
    for test_input, test_label in test_dataset:    
        test_accuracy = accuracy_fn(network, test_input, test_label)

    print("test_Accuracy: %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
 [!] Load failed...
Epoch: [ 0] [    0/  468] time: 0.4613, train_loss: 2.20311832, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [    1/  468] time: 0.5605, train_loss: 2.19566917, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [    2/  468] time: 0.6504, train_loss: 2.21879148, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [    3/  468] time: 0.7470, train_loss: 2.15959549, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [    4/  468] time: 0.8405, train_loss: 1.97626138, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [    5/  468] time: 0.9272, train_loss: 1.95614123, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [    6/  468] time: 1.0144, train_loss: 1.90386617, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [    7/  468] time: 1.1102, train_loss: 1.89223790, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [    8/  468] time: 1.1943,

Epoch: [ 0] [   76/  468] time: 7.3912, train_loss: 0.51771450, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [   77/  468] time: 7.4826, train_loss: 0.50326508, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [   78/  468] time: 7.5762, train_loss: 0.53750086, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [   79/  468] time: 7.6694, train_loss: 0.70644373, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [   80/  468] time: 7.7663, train_loss: 0.61932659, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [   81/  468] time: 7.8615, train_loss: 0.57251751, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [   82/  468] time: 7.9546, train_loss: 0.59150213, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [   83/  468] time: 8.0413, train_loss: 0.52795160, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [   84/  468] time: 8.1307, train_loss: 0.40867579, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0

Epoch: [ 0] [  152/  468] time: 14.5543, train_loss: 0.48120254, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [  153/  468] time: 14.6477, train_loss: 0.47884130, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [  154/  468] time: 14.7349, train_loss: 0.43127823, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [  155/  468] time: 14.8324, train_loss: 0.23675634, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [  156/  468] time: 14.9228, train_loss: 0.38531780, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [  157/  468] time: 15.0062, train_loss: 0.38255471, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [  158/  468] time: 15.0898, train_loss: 0.41109622, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [  159/  468] time: 15.1739, train_loss: 0.42027688, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [  160/  468] time: 15.2585, train_loss: 0.53815240, train_accuracy: 1.0000, test_Accuracy: 1.0000
E

Epoch: [ 0] [  227/  468] time: 21.5776, train_loss: 0.35163873, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [  228/  468] time: 21.6655, train_loss: 0.34099865, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [  229/  468] time: 21.7572, train_loss: 0.33685958, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [  230/  468] time: 21.8555, train_loss: 0.24598911, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [  231/  468] time: 21.9432, train_loss: 0.36188281, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [  232/  468] time: 22.0314, train_loss: 0.31270775, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [  233/  468] time: 22.1303, train_loss: 0.32415879, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [  234/  468] time: 22.2171, train_loss: 0.33054602, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [  235/  468] time: 22.3096, train_loss: 0.34132335, train_accuracy: 1.0000, test_Accuracy: 1.0000
E

Epoch: [ 0] [  301/  468] time: 28.6555, train_loss: 0.43463019, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [  302/  468] time: 28.7515, train_loss: 0.30198228, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [  303/  468] time: 28.8698, train_loss: 0.34957704, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [  304/  468] time: 28.9707, train_loss: 0.31375766, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [  305/  468] time: 29.0619, train_loss: 0.36892399, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [  306/  468] time: 29.1602, train_loss: 0.42311093, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [  307/  468] time: 29.2767, train_loss: 0.27505761, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [  308/  468] time: 29.3683, train_loss: 0.46510690, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [  309/  468] time: 29.4563, train_loss: 0.25891322, train_accuracy: 1.0000, test_Accuracy: 1.0000
E

Epoch: [ 0] [  376/  468] time: 35.8024, train_loss: 0.35336173, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [  377/  468] time: 35.9004, train_loss: 0.27641213, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [  378/  468] time: 35.9907, train_loss: 0.29830003, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [  379/  468] time: 36.0796, train_loss: 0.37520391, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [  380/  468] time: 36.1666, train_loss: 0.34037068, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [  381/  468] time: 36.2562, train_loss: 0.16801693, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [  382/  468] time: 36.3433, train_loss: 0.32324839, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [  383/  468] time: 36.4327, train_loss: 0.29205310, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [  384/  468] time: 36.5214, train_loss: 0.34295338, train_accuracy: 1.0000, test_Accuracy: 1.0000
E

Epoch: [ 0] [  451/  468] time: 42.6143, train_loss: 0.17902043, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [  452/  468] time: 42.7042, train_loss: 0.36342683, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [  453/  468] time: 42.7983, train_loss: 0.24442643, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [  454/  468] time: 42.8975, train_loss: 0.22170806, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [  455/  468] time: 42.9856, train_loss: 0.07666700, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [  456/  468] time: 43.0805, train_loss: 0.36116350, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [  457/  468] time: 43.1746, train_loss: 0.28540155, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [  458/  468] time: 43.2625, train_loss: 0.28508002, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [  459/  468] time: 43.3522, train_loss: 0.23948501, train_accuracy: 1.0000, test_Accuracy: 1.0000
E